*2021/01/10 Sun*

# 6-1. C++ 표준 문자열 & 부모의 것을 물려쓰자 - 상속

## std::string

In [2]:
#include <iostream>
#include <string>

int main() {
    std::string s = "abc";  // const char * 생성자 호출
    std::cout << s << std::endl;
    return 0;
}

## 상속

In [ ]:
// 기반(부모) 클래스
class Base {
    std::string s;
    
public:
    Base() : s("기반") { std::cout << "기반 클래스" << std::endl; }
    
    void what() { std::cout << s << std::endl; }
};

// 파생(자식) 클래스 : C++의 경우, 여러 명의 부모를 가질 수 있음. 
class Derived : public Base {  // public 형식으로 상속받겠다.
    std::string s;  // 원래의 s를 가림

public:
    Derive() : Base(), s("파생") {  // 기반의 생성자 먼저 호출. 명시적으로 호출하지 않으면 디폴트 생성자 호출됨.
        std::cout << "파생 클래스" << std::endl;
        
        what();
    }
};

## 새로운 친구 protected

```protected``` : 상속받는 클래스에서는 접근 가능하고, 그 외의 기타 정보는 접근 불가능.

In [ ]:
class Base {
    std::string parent_string;
    // ...
};

class Derived : public Base {
public:
    Derived() : child_string("파생"), Base() {
        // ...
        parent_string = "바꾸기"  // error : cannot access private member declared in class 'Base'
    }
};

비유하자면,

* ```private```  : (부모님들한테 안 가르쳐 주는) 자신만의 비밀번호
* ```protected``` : 집 현관문 비밀번호 (가족들은 알지만 그 외의 사람들은 접근 불가)
* ```public``` : 집 주소 (가족뿐만이 아니라 다른 사람들도 알 수 있음)

3단계의 접근 지시자로 멤버의 접근 허용 범위를 지정할 수 있다. 따라서 위에서는 ```parent_string```을 ```protected```로 접근 허용 범위를 바꾸면 잘 실행됨.

#### 그래서 ```public``` 형식으로 상속받겠다는 뜻이 뭐냐?

```
class Derived : public Base { ...
```

* ```public``` 형태로 상속받으면 기반 클래스의 접근 지시자가 파생 클래스에서도 그대로 유지.
* ```protected```로 하면 ```public```은 ```protected```로 내려오고 나머지는 그대로 유지.
* ```private```로 하면 모두 ```private```으로 내려옴.

# 6-2. 가상(virtual) 함수와 다형성

## 'is-a'와 'has-a'

상속을 도입한 것은 단순히 똑같은 코드를 또 쓰는 것을 막기 위한 것이 아님. 상속이라는 기능을 통해 객체 지향 프로그래밍에서 추구하는 실제 객체의 추상화를 좀 더 효과적으로 할 수 있게 되었다.

C++에서는 상속이란 것을 도입해서, 클래스 사이에 관계를 표현할 수 있게 되었음.

```
class Manager : public Employee
```

```Manager``` 클래스는 ```Employee```의 기능을 모두 수행할 수 있기 때문에, ```Manager``` is a ```Employee```!

모든 상속 관계는 ```is-a``` 관계라고 볼 수 있음.

![is-a](figures/Screen%20Shot%202021-01-10%20at%208.57.50%20PM.png)

한편, 어떤 클래스들 간에는 ```is-a``` 대신에 ```has-a``` 관계가 성립하기도 함.
> 자동차 has a 엔진

## (다시 보는) 오버라이딩

In [ ]:
Base p;  // 기반 클래스
Derived c;  // 파생 클래스

Base *p_c = &c;  // 업 캐스팅
Derived *p_p = &p;  // 다운 캐스팅 -> error : cannot convert from 'Base *' to 'Derived *'

컴파일러는 함부러 다운 캐스팅하는 걸 막는다. 그런데,

In [ ]:
Base *p_p = &c;
Derived *p_c = p_p;  // 동일한 error

이것도 막음. 실제로는 문제가 없는데도.

이럴 때는 ```Derived *p_c = static_cast<Derived *>(p_p);```와 같이 강제로 타입 변환을 하면 되지만, 실제로 오류가 날 만한 일이었다면 런타임 오류가 발생. 그리고 이렇게 하면 컴파일 타임에서 오류를 찾아내기 매우 힘들기 때문에, 이러한 방법은 매우 권장하지 않음.

→ **```dynamic_cast```!**
```
Derived *p_c = dynamic_cast<Derived *>(p_p);
```
이렇게 하면, 실제로 불가능한 상황을 컴파일러가 감지해서 캐스팅할 수 없다는 컴파일 오류를 띄워준다.

## virtual 키워드

업 캐스팅한 상태에서 멤버 함수 호출하면 기반 클래스의 멤버 함수가 호출됨. 오버라이딩되어야 하는 경우에도.

→ 해결 방법 : ```virtual``` 키워드 -> **가상 함수**

In [ ]:
class Base {
public:
    virtual void what() {}  // virtual 키워드를 붙여 가상 함수로 만들었다.
};

class Derived : public Base {
public:
    void what() {}
};

int main() {
    Derived c;
    Base *p_c = &c;
    
    p_c->what();  // 파생 클래스의 what() 호출. (기반 클래스에서 해당 함수에 virtual 키워드 썼기 때문에.)
}

- ```virtual``` 키워드는 **동적 바인딩**과 연결.
    - dynamic binding : 어떤 함수가 실행될지 정하는 것을 런타임 때 하는 것.
    - static binding : 컴파일 타임에 어떤 함수가 호출될 때 정해지는 것. 여태까지 알고 있던 것.

> "흠, p_c는 Base 포인터니까 Base의 what()을 실행해야지"

> "어, 근데 what()이 virtual이네?"

> "잠깐, 이거 실제 Base 객체 맞어? 아니네 Derived 객체네"

> "그럼 Derived의 what을 실행해야지"

## override 키워드

파생 클래스에 붙이는 것. 실수로 오버라이드를 하지 않는 경우를 막을 수 있음.

In [ ]:
class Base {
    virtual void incorrect() {}
};

class Derived {
    void incorrect() const {}  // const???
    
    // 이건 상수 함수이므로 오버라이드된 함수가 아닌, 다른 함수로 간주.
    
    // 따라서 아래와 같이 override 키워드를 붙이면,
    
    void incorrect() const override {}  // error : ... but does not override
    
    // 컴파일 오류가 정상적으로 발생.
}

## 즉, 다형성!

하나의 메서드를 호출했음에도 불구하고 여러 가지 다른 작업들을 하는 것.

## 생각해보기

그렇다면 프로그램 내부적으로 virtual 함수들은 어떻게 처리될까요? 즉, 이 포인터가 어떠한 객체를 가리키는지 어떻게 알 수 있을까요? (난이도 : 上)

> 객체가 생성되면 객체 고유의 가상 함수 테이블이 만들어지고, 이 테이블을 참조하여 함수를 호출.

> 자식 객체가 생성되면 부모 객체의 vtable을 복사하고, 오버라이딩된 함수만 주소가 새로 업데이트된다고 한다. 그리고 만약 자식 클래스에 부모에 없는 새로운 가상 함수를 추가할 경우, 객체의 vtable 마지막 부분에 추가된다.

# 6-3. 가상 함수와 상속에 관련한 잡다한 내용들

## virtual 소멸자

In [ ]:
#include <iostream>

class Parent {
 public:
  Parent() { std::cout << "Parent 생성자 호출" << std::endl; }
  ~Parent() { std::cout << "Parent 소멸자 호출" << std::endl; }
};
class Child : public Parent {
 public:
  Child() : Parent() { std::cout << "Child 생성자 호출" << std::endl; }
  ~Child() { std::cout << "Child 소멸자 호출" << std::endl; }
};
int main() {
  std::cout << "--- 평범한 Child 만들었을 때 ---" << std::endl;
  { Child c; }  // 블록 벗어나면 지역 변수 소멸
  std::cout << "--- Parent 포인터로 Child 가리켰을 때 ---" << std::endl;
  {
    Parent *p = new Child();
    delete p;
  }
}

→ 출력 :

```
--- 평범한 Child 만들었을 때 ---
Parent 생성자 호출
Child 생성자 호출
Child 소멸자 호출
Parent 소멸자 호출
--- Parent 포인터로 Child 가리켰을 때 ---
Parent 생성자 호출
Child 생성자 호출
Parent 소멸자 호출
```
* 자식 객체의 생성자/소멸자 호출 순서 : 
> ```부모 생성자``` -> ```자식 생성자``` -> ```자식 소멸자``` -> ```부모 소멸자```

위 케이스의 경우는 잘 수행되었는데, 아래 케이스의 경우는 그렇지 않음(```Parent *p = new Child(); delete p;```)

→ 메모리 누수!

→ 부모의 소멸자를 ```virtual```로 만들어 버리자.

자식은 부모를 상속받는 것을 알고 있지만, 부모는 자식이 있는지 없는지 모름.
따라서 자식 소멸자를 호출해줄 수 없어서 이런 일이 벌어지는 것.

따라서, 상속될 여지가 있는 클래스들은 **반드시 소멸자를 ```virtual```로 만들어 주어야** 나중에 문제가 발생할 여지가 없다.

## 레퍼런스도 된다

가상 함수에 대해서 포인터를 예제로 들었지만, 레퍼런스여도 똑같이 동작.

In [ ]:
// ... (Parent의 show()를 virtual, Child의 show()를 override)

void test(Base& a) { a.show(); }

int main() {
    Parent a;
    Child b;
    test(a);  // Parent의 show();
    test(b);  // Child의 show();
}

## 가상 함수의 구현 원리

* 의문

> 그냥 그럼 모든 함수들을 ```virtual```로 만들어 버리면 안되나?

사실, 모든 함수들을 그렇게 만들어버린다고 해도 문제될 것이 전혀 없음. **실제로 자바는 모든 non-static 함수가 디폴트로 ```virtual``` 함수로 선언.**

> java는 static, private, final 메소드는 static binding을 하고 그 외 모든 함수는 dynamic binding(virtual)인 것으로 알고 있습니다.

그러나, 가상 함수를 사용하면 약간의 **오버헤드**가 존재한다.

In [ ]:
class Parent {
 public:
  virtual void func1();
  virtual void func2();
};
class Child : public Parent {
 public:
  virtual void func1();
  void func3();
};

![vtable](figures/Screen%20Shot%202021-01-10%20at%2010.27.59%20PM.png)

위와 같이, 가상 함수 테이블은 전화번호부라 생각하면 된다. (자식 클래스의 것에 func2()이 있는 건, 부모의 vtable을 복제해서 만들었기 때문.)

가상 함수면 가상 함수 테이블을 거쳐 두 단계에 걸쳐 함수를 호출. 소프트웨어적으로 동적 바인딩을 구현하게 됨. 그래서 더 오래 걸림.

차이는 극히 미미하지만, 최적화가 매우 중요한 분야에서는 이를 감안할 필요가 있으므로 디폴트로 가상 함수가 되도록 설정하지 않은 것.

## 순수 가상 함수(pure virtual function)와 추상 클래스(abstract class)

In [ ]:
class Animal {
public:
    Animal() {}
    virtual ~Animal() {}
    
    virtual void speak() = 0;  // 순수 가상 함수 (= 추상 메소드) : = 0;을 붙이면 됨.
};

가상 함수에 ```= 0;```을 붙이면 순수 가상 함수(무엇을 하는지 정의되어 있지 않는 함수). 즉, 반드시 오버라이딩되어야만 하는 함수.

순수 가상 함수는 본체가 없으므로 이 함수를 호출하는 것도 불가능하고, 이러한 함수가 포함된 ```Animal``` 객체를 생성하는 것 또한 불가능.

그래서 이런 클래스는 **추상 클래스**라고 함.

*(참고 : ```private``` 안에 순수 가상 함수 정의해도 문제될 것은 없음. 오버라이드 안 된다는 건 아니니까. 다만 자식 클래스에서 호출을 못할 뿐임.)*

*(확인은 안 해봤으나, 오버라이딩 메서드는 부모에서 지정한 접근 지시자와 동일해야 하는 듯.)*

추상 클래스는 "이 기능은 일반적인 상황에서 만들기 힘드니, 너가 직접 특수화되는 클래스에 맞추어 만들어서 써라."라고 말해주는 것과 같음.

그리고 추상 클래스는 객체는 생성할 수 없어도 가리키는 포인터는 문제 없이 만들 수 있음.

## 다중 상속(multiple inheritance)

In [ ]:
class C : public B, public A {
public:
    int c;
};

```B```를 ```A```보다 먼저 상속했으므로, ```B```의 생성자가 먼저 호출됨. 이 순서는 오직 상속하는 순서에만 좌우된다.

* 주의할 점

In [ ]:
class A {
public:
    int a;
};

class B {
public:
    int a;
};

class C : public B, public A {};

int main() {
    C c;
    c.a = 3;  // error : ambiguous access of 'a'
}

특히, **다이아몬드 상속 문제**!

![다이아몬드 상속](figures/Screen%20Shot%202021-01-10%20at%2011.38.01%20PM.png)

```Me```의 입장에서는, 다른 것에서 모호성이 없더라도 ```Human```의 모든 내용은 무조건 중복되는 문제가 발생.

* 해결 방법? → ```virtual```로 상속!

In [ ]:
class Human {
public:
    // ...
};
class HandsomeHuman : public virtual Human {
    // ...
};
class SmartHuman : public virtual Human {
    // ...
};
class Me : public HandsomeHuman, public SmartHuman {
    // ...
};

```Human```을 ```virtual```로 상속받으면(가상 상속), 다중 상속 시에도 컴파일러가 언제나 ```Human```을 한번만 포함하도록 지정할 수 있음. 물론 이 때에도 ```Human```의 생성자는 호출되는 것.

단, 다중 상속이 아님. 다중 상속을 이용해서 해결해야 될 것 같은 문제도 알고 보면 단일 상속을 통해 해결할 수 있는 경우도 있음.

추가로 절대적인 예는 아니나, 예로 들어 차량이 있고, 그 차량이 작동하는 환경의 종류가 N개(땅, 우주, ...) 중 1개, 동력원의 종류가 M개(석유, ...) 중 1개라 하자.

세부적인 경우의 수를 구분하여 멤버 함수들을 오버라이드해야 하고, 그것들을 모두 ```Vehicle&```로 참조해야 할 때, 이러한 클래스는 크게 3가지 방법으로 디자인할 수 있음.
* 브릿지 패턴(bridge pattern)
* 중첩된 일반화 방식(nested generalization)
* 다중 상속